1\. Write a function that converts number representation (bin<->dec<->hex)

In [96]:
# decimale-->bin
def convert_dec_bin (x):
    b=[]
    a=[]
    n=0
    a.append(x)
    while(x/2**n>=1):
        a.append((a[n]-(a[n]%2))/2)
        b.append(int(a[n]%2))
        n=n+1
    b=b[::-1]
    St = "".join([str(_) for _ in b])
    return St 

y=convert_dec_bin(70)
print(y)

# bin--->decimale
def convert_bin_dec(x):
    n=len(x)
    s=[int(y) for y in x]
    d=0
    for i in range (n):
        d=d+s[i]*2**(len(x)-1-i)
    return d

e=convert_bin_dec(y)
print(e)

# decimale-->hex
def convert_dec_hex(x) :
    b=[]
    a=[]
    n=0
    a.append(x)
    while(x/16**n>=1):
        a.append((a[n]-(a[n]%16))/16)
        if (int(a[n]%16) < 10):
            b.append(int(a[n]%16))
        elif(int(a[n]%16) == 10):
            b.append('A')
        elif(int(a[n]%16) == 11):
            b.append('B')    
        elif(int(a[n]%16) == 12):
            b.append('C')
        elif(int(a[n]%16) == 13):
            b.append('D')
        elif(int(a[n]%16) == 14):
            b.append('E')
        elif(int(a[n]%16) == 15):
            b.append('F')
        n=n+1
    b=b[::-1]
    St = "".join([str(_) for _ in b])
    return St

d = convert_dec_hex(11184810)
print(d)

# hex--->decimale
def convert_hex_dec(x) :
    n=len(x)
    s=[y for y in x]
    d=0
    for i in range (n):
        if (s[i].isnumeric()):
            d=d+s[i]*16**(len(x)-1-i)
        elif( s[i] == 'A'):
            d=d + 10 *16**(len(x)-1-i)
        elif( s[i] == 'B'):
            d=d + 11 *16**(len(x)-1-i)
        elif( s[i] == 'C'):
            d=d + 12 *16**(len(x)-1-i)
        elif( s[i] == 'D'):
            d=d + 13 *16**(len(x)-1-i)
        elif( s[i] == 'E'):
            d=d + 14 *16**(len(x)-1-i)
        elif( s[i] == 'F'):
            d=d + 15 *16**(len(x)-1-i)
    return d

print(convert_hex_dec(d))

1000110
70
AAAAAA
11184810


2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [112]:
x='11011010000111101101101110111111'
m = int(x[1:9],2) 
m = 1 + m*10**-len(str(m))
print(m)
print(x[10:])
print(int(x[10:], 2))
def convert ( x ) :
    a = [i for i in x if ((i == '0') or (i =='1'))]
    b = 0
    if( (len(x) == 32) and len(a) == 32 ):
        # exponent
        e = int(x[1:9],2) - 127
        
        # mantissa
        m = 1 + int(x[1:9],2)*10**-len(str(int(x[10:], 2)))
        b = m * 2 ** e
        if (a[0]== '1') : b = -1+b
    else : print(" Invalid input ")
    return  b

print(convert(x))

1.18
0111101101101110111111
2022335
9007361384327576.0


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [240]:
a = 1
b = 1
underflow = []
overflow  = []
i = 0

while ( a > 0 ):
    a = a / ( 2**i )
    underflow.append(a)
    i = i + 1
    
print("Underflow:", underflow[i-2])

i = 0
overflow.append(0)
# I decided to stop when the computation can be run
# in few seconds
while ( i < 2000):
    b = b *(2**i) 
    overflow.append(b)
    i = i + 1
    
print("Overflow:" , overflow[i-1] )

Underflow: 2.716154612436e-312
Overflow: ...


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [113]:
x = 1
delta = 10**-2
i = 2
while (x + delta > 1):
    delta = 10**-i
    i = i +1
    #print(x + delta)
    
print ("Machine precision: " , delta)

Machine precision:  1e-16


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [221]:
import math

def f_1( a, b, c ):
    x_1 = (-b + math.sqrt(b**2 - (4*a*c)))/(2*a)
    x_2 = (-b - math.sqrt(b**2 - (4*a*c)))/(2*a)
    return x_1, x_2

# a)
a = 0.001
b = 1000
c = 0.001
x_1 , x_2 = f_1( a, b, c )
print("a) x_1=",x_1,"x_2=", x_2)

# b)
def f_2( a, b, c ):
    x_1 = ( - 4*a*c)/((2*a)*(-b - math.sqrt(b**2 - (4*a*c))))
    x_2 = ( - 4*a*c)/((2*a)*(-b + math.sqrt(b**2 - (4*a*c))))
    return x_1, x_2

x_3 , x_4 = f_2( a, b, c )
print("b) x_1=",x_3,"x_2=", x_4)

# c)
def f_3( a, b, c ):
    x_1 = ( - 4*a*c)/((2*a)*(-b - math.sqrt(b**2 - (4*a*c))))
    x_2 = (-b - math.sqrt(b**2 - (4*a*c)))/(2*a)
    return x_1, x_2

x_5 , x_6 = f_3( a, b, c )
print("c) x_1=",x_5,"x_2=", x_6)


a) x_1= -9.999894245993346e-07 x_2= -999999.999999
b) x_1= 1.0000000000009999e-06 x_2= 1000010.5755125057
c) x_1= 1.0000000000009999e-06 x_2= -999999.999999


6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [216]:
def f( x ):
    return x * ( x - 1 )

def d( x ):
    return ( (2* x) - 1 ) 

# a) derivatice
delta = 10**(-2)
x = 1
derivative_1 = ( f( x + delta ) - f( x ) )/ delta 
derivative_2 = d(x)
print("Real value:",derivative_2)
print("Computed value (delta =", delta , "): ", derivative_1)

# b)
for i in range ( 3 , 15) :
    if ((i % 2)==0) : 
        delta = 10**(-i)
        derivative_1 = ( f( x + delta ) - f( x ) )/ delta 
        derivative_2 = d(x)
        print("Computed value (delta =", delta , "): ", derivative_1)

Real value: 1
Computed value (delta = 0.01 ):  1.010000000000001
Computed value (delta = 0.0001 ):  1.0000999999998899
Computed value (delta = 1e-06 ):  1.0000009999177333
Computed value (delta = 1e-08 ):  1.0000000039225287
Computed value (delta = 1e-10 ):  1.000000082840371
Computed value (delta = 1e-12 ):  1.0000889005833413
Computed value (delta = 1e-14 ):  0.9992007221626509


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [208]:
#a) 
import math

def Riemann (N) :
    A = 0
    for i in range (int(- N/2) , int(N/2)):
        y = math.sqrt(1 - (2*i/N)**2) 
        A = A + ( y * 2 / N)  
    return  A

A = math.pi / 2
N=10**2
print ("Distance between true and computed value, ( N =",N,"):", A-Riemann(N))

#b) 1 sec
N=10**6
print ("Distance between true and computed value, ( N =",N,"):", A-Riemann(N))

# 1 min
N=10**8
print ("Distance between true and computed value, ( N =",N,"):", A-Riemann(N))

Distance between true and computed value, ( N = 100 ): 0.0016620712456461018
Distance between true and computed value, ( N = 1000000 ): 1.663168713861296e-09
Distance between true and computed value, ( N = 100000000 ): 1.6127099655705024e-12
